In [50]:
import rexerclient
from rexerclient import rql
from rexerclient.rql import *
from rexerclient import client
from rexerclient.models import action
import time
from rexerclient import models
from rexerclient.models import value

In [57]:
url = 'http://172.31.28.201/data'
c = client.Client(url)

In [70]:
# for entity which is of type "video" and video_id: 12312, set "category_id" to be 31
# c.set_profile("user", 12312, "age", 31)

pi = models.profile.ProfileItem()
pi.Oid = 12312
pi.OType = 'video'
pi.Key = "category"
pi.Value.CopyFrom(value.Int(31))
pi.CustID = 1
c.set_profile(pi)

In [63]:
req = models.profile.ProfileItem()
req.Oid = 12312
req.OType = 'user'
req.Key = 'age'
req.CustID = 1
c.get_profile(pi)

Int: 31

In [64]:
a = action.Action()
a.ActionType = 'like'
a.ActorType = 'user'
a.ActorID = 12312
a.TargetType = 'video'
a.TargetID = 82342
a.RequestID = 523423 # request is the feed load in which this action took place
a.Timestamp = int(time.time()) # seconds since epoch when this action took place
a.CustID = 1  # ignore

# does the API for logging actions need to be in Python or Go?
# answer: needs both Python + go clients

In [65]:
c.log(a)


In [66]:
c.fetch(action.ActionFetchRequest())

[ActionID: 1
 CustID: 1
 ActorID: 12312
 ActorType: "user"
 TargetID: 82342
 TargetType: "video"
 ActionType: "like"
 Timestamp: 1643689125
 RequestID: 523423,
 ActionID: 2
 CustID: 1
 ActorID: 12312
 ActorType: "user"
 TargetID: 82342
 TargetType: "video"
 ActionType: "like"
 Timestamp: 1643689149
 RequestID: 523423]

In [68]:

    

# Given a uid, what is their average watch time on any video they might have seen?
# when you log the action, you store video watch time in a field called 'action_value'
    
# this following query measures the total time spent by the user on any video in last x days
q1 = Var('args').actions.apply(
    Ops.std.filter(where=at.action_type == String('video_view'))).apply(        # filter to only 'like' actions
    Ops.std.addColumn(name='key', value=List(at.actor_id), increment=at.action_value)   # counter key is (uid)
)
options = models.aggregate.AggOptions()
options.duration = 3600*24*9  # 3 day rolling counters
c.store_aggregate('rolling_counter', 'user_video_like', q, options)


# measures total videos viewed by the user uid in the last x days
q2 = Var('args').actions.apply(
    Ops.std.addProfileColumn(otype='video', oid=at.target_id, key='category').apply(
    Ops.std.addProfileColumn(otype='user', oid=at.user_id, key='age').apply(
    Ops.std.filter(where=at.action_type == String('video_view'))).apply(        # filter to only 'like' actions
    Ops.std.addColumn(name='key', value=List(at.actor_id, at.gender, at.age, at.group_id), increment=1)   # counter key is (uid)
)
options = models.aggregate.AggOptions()
options.duration = 3600*24*9  # 3 day rolling counters
c.store_aggregate('rolling_counter', 'user_video_like', q, options)

find(q1) / find(q2)


In [69]:
req = models.aggregate.GetAggValueRequest()
req.agg_type = 'rolling_counter'
req.agg_name = 'user_video_like'
req.key.CopyFrom(value.List(value.Int(12312), value.Int(82342)))
c.aggregate_value(req)

Int: 0